# El pole dance y la diagonalización

Hace poco me enteré de que una miga va a clases de pole dance todos los lunes.

Yo pensaba que era una cosa de los bares de las películas. De esas en las que dos policías atraviesan el humo en busca de algún mafioso que siempre esta al otro lado de la mujer que baila en la barra.

Ese día aprendí que resulta que es un ejercicio bastante copleto. Y divertido.

A diferencia de la diagonalización, pensarás.

Diagonalizar es pasarte la vida calculando determinantes, para que al final tus autovalores den cosas como $13.007425124$ y no saber qué has hecho mal para que no te dé un número bonito.

Pero lo peor-peor, es que es inútil para un químico.

Nada que ver con el pole dance, que mejora tu salud.

<br>

Pues hoy me toca convencerte de lo contrario. Cuando acabes de leer esto entenderás dos cosas:

1. Que cada vez que te digan que algo es diagonalizable, debes dar gracias a Dios (o a tu signo del zodíaco, me da igual tu religión).

2. Que la diagonalización y el pole dance tienen tanto que ver, que su relación con la química sale sola.

Primero, déjame recordarte

## Qué hace especial a las matrices diagonalizables

En las matrices diagonalizables puedes confiar. Traen bienestar al mundo.

Porque si $A$ es diagonalizable hay una base en la que se comporta como una multiplicación por números, es diagonal.

Es decir, existe un marco de referencia en el que $A$ es mucho más simple.

**Poniéndonos un poco más matemáticos:**

Una matriz $A$ de dimensiones $n\times n$ es diagonalizable, si se cumple

$$
A x_i = \lambda_i x_i
$$

para una base de vectores $(x_1, ...,x_n)$ y unos valores $\lambda_1,...,\lambda_n$.

$x_i$ son los autovectores (eigenvectors)

$\lambda_i$ son los autovalores (eigenvalues)

<br>
<br>

Voy a definir un par de matrices y ya empiezo con el pole dance.

<br>

La matriz que tiene los autovalores en la diagonal:

$$
D = 
\begin{pmatrix}
\lambda_1 & & & & \\
 & \lambda_2 & & & \\
 & & \ddots & & \\
 & & & \lambda_n
\end{pmatrix}
$$

<br>

La matriz que tiene los autovectores en las columnas:

$$
X =\begin{pmatrix}
x_1 & x_2 & ... & x_n
\end{pmatrix}
$$

<br>

Con estas dos matrices, la ecuación $A x_i = \lambda_i x_i$ se puede escribir como

$$
A X = X D
$$

Como las columnas de $X$ son vectores linealmente independientes existe su inversa $X^{-1}$, la multiplicamos por la derecha y nos queda:

$$
A = X D X^{-1}
$$

¡Tarán!

<br>

Además, si se puede escoger la matriz $X$ para que sea [unitaria](https://es.wikipedia.org/wiki/Matriz_unitaria) (es decir $X^{\dagger}=X^{-1}$), entonces $A$ es diagonalizable por unitarias y la base $X$ es ortonormal.

En otra ocasión veremos por qué eso es digno de un buen café a mi salud.

<br>

Todo en su sitio.

Vamos con lo importante.

## Una caja bailando pole dance

Pongamos que tienes una caja dando vueltas al rededor de un eje.

Si quieres, la caja es la persona bailando y el eje es la barra.

Supón que quieres calcular su momento angular.

Si eres bailarín de pole dance esto te interesa, para saber con qué fuerza tienes que agarrarte a la barra. Ya ves.

Para que nos entendamos, voy a empezar por dibujarte la caja.

(no re preocupes por entender el código aún)

In [1]:
%matplotlib notebook

Aquí defino las dimensiones de la caja, el vector de rotación, calculamos la matriz de inercia y el momento angular inicial:

In [2]:
import numpy as np
from scipy.spatial.transform import Rotation as R

# Define the vertices of the cube
a, b, c = 3, 2, 1
vertices = np.array([[-a/2, -b/2, -c/2],
                     [a/2, -b/2, -c/2],
                     [a/2, b/2, -c/2],
                     [-a/2, b/2, -c/2],
                     [-a/2, -b/2, c/2],
                     [a/2, -b/2, c/2],
                     [a/2, b/2, c/2],
                     [-a/2, b/2, c/2]])

# Define the edges of the cube
edges = [(0,1), (1,2), (2,3), (3,0),
         (4,5), (5,6), (6,7), (7,4),
         (0,4), (1,5), (2,6), (3,7)]

# Define the rotation vector
omega = np.array([1,-3,1]) / np.sqrt(12)

# Define the mass
M = 2 # kg

# Compute the initial matrix of inertia
I0 = 1/12 * M * np.diag([b**2+c**2, a**2+c**2,a**2+b**2])

# Compute the initial angular momentum L0
L0 = I0 @ omega

Aquí la hago rotar con mi brujería pytónica:

In [3]:
import matplotlib.pyplot as plt
from matplotlib import animation

from IPython import display
plt.rcParams['animation.ffmpeg_path'] = 'utils/ffmpeg'
plt.ioff()

# Animate the cube by rotating it
def update_mesh(i, lines, markers, arrows):
    theta = i * np.pi / 180
    r = R.from_rotvec(theta * omega)

    rotated_vertices = vertices @ r.as_matrix()
    for line, edge in zip(lines, edges):
        line.set_data(rotated_vertices[edge,:2].T)
        line.set_3d_properties(rotated_vertices[edge,2].T)
        
    for marker, vert in zip(markers, rotated_vertices):
        marker.set_data([vert[0]],[vert[1]])
        marker.set_3d_properties(vert[2])
        
    if i % 50 == 0:
        for arrow in arrows:
            L = r.as_matrix() @ L0
            ax.quiver(0,0,0,L[0],L[1],L[2], alpha=.5)

    return lines

In [4]:
fig = plt.figure()
ax = fig.add_subplot(projection="3d")

# Define the initial lines for the edges
lines = [ax.plot(*vertices[edge,:].T,
                 color='black')[0] for edge in edges]

markers = [ax.plot(*vert,
                 color='black', marker='.',
                markersize=10)[0] for vert in vertices]

arrows = [ax.quiver(0,0,0,L0[0],L0[1],L0[2], alpha=0.5)]

# Setting the axes properties
ax.set(xlim3d=(-2, 2), xlabel='X')
ax.set(ylim3d=(-2, 2), ylabel='Y')
ax.set(zlim3d=(-2, 2), zlabel='Z')

# Set up the animation
anim = animation.FuncAnimation(
    fig, update_mesh, 360,
    fargs=(lines,markers, arrows), interval=25)

video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()

Para calcular el momento angular necesitas saber dos cosas:

1. El **vector de rotación**

$$\mathbf{\Omega} = 
\begin{pmatrix}
\omega_x\\ \omega_y \\ \omega_z
\end{pmatrix}
$$

2. La **matriz de inercia**

$$
I = \begin{pmatrix}
I_{xx} & I_{xy} & I_{xz}\\
I_{yx} & I_{yy} & I_{yz}\\
I_{zx} & I_{zy} & I_{zz}
\end{pmatrix}
$$

Entonces el **momento angular** es

$$
\mathbf{L} = I \mathbf{\Omega}
$$

$\mathbf{\Omega}$ está en la variable `omega`

In [5]:
omega

array([ 0.28867513, -0.8660254 ,  0.28867513])

$I$ en $t=0$ es la de un [paralelepípedo con los ejes alineados](https://es.wikipedia.org/wiki/Anexo:Tensores_de_momento_de_inercia_3D), en la variable `I0`

In [6]:
I0

array([[0.83333333, 0.        , 0.        ],
       [0.        , 1.66666667, 0.        ],
       [0.        , 0.        , 2.16666667]])

Así que $\mathbf{L}$ es

In [7]:
L0 = I0 @ omega
L0

array([ 0.24056261, -1.44337567,  0.62546279])

Fácil.

En $t=0$ la matriz $I$ es diagonal. Qué fácil.

Pero atento a esto: _¿Qué pasa si $t>0$?_

Pasa que la caja ha girado.

Así que si describimos $I$ en los ejes $x,y,z$ ya NO será diagonal.

·\_·


<br>

Para todo $t$ el vector $\mathbf{\Omega}$ es el mismo, porque la barra de pole dance marca un eje fijo.

Para calcular $I$ en el instante $t$ tenemos que rotarla:

$I(t)=R(t\mathbf{\Omega}) I_0 R(t\mathbf{\Omega})^T$

Aquí $R(t\mathbf{\Omega})=R_{\mathbf{u}}(\theta)$ es una rotación de ángulo $\theta = \Vert t \mathbf{\Omega} \Vert$ en la dirección definida por el vector unitario $\mathbf{u}=\frac{\mathbf{\Omega}}{\Vert \mathbf{\Omega}\Vert}$.

Puedes ver cómo se obtiene $R_{\mathbf{u}}(\theta)$ en la entrada de Wikipedia sobre [la fórmula de rotación de Rodrigues](https://es.wikipedia.org/wiki/F%C3%B3rmula_de_rotaci%C3%B3n_de_Rodrigues) (el tío era francés).

Pero no es necesario.

En Python puedes calcular esta matriz usando la clase `Rotation` de `scipy.spatial.transform` y el método `r=R.from_rotvec(theta * u)`. 

Esto en realidad devuelve un objeto `r` de tipo `Rotation`, usa `r.as_matrix()` para que te dé la matriz correspondiente.

Es fácil ver que si algo gira sobre sí mismo no cambia, es decir 

$$R(t\mathbf{\Omega})\mathbf{\Omega}=R(t\mathbf{\Omega})^T\mathbf{\Omega}=\mathbf{\Omega}$$

Esto implica que $\mathbf{L}$ va rotando

$$
\mathbf{L}(t) = R(t\mathbf{\Omega}) \mathbf{L}_0
$$

(como se muestra en la fabulosa animación de la caja, que me ha quedado dpm y la tengo que poner otra vez)

In [8]:
display.display(html)
plt.close()

## Volvamos a lo de la diagonalización

Bueno.

Como puedes ver para tu ordenador es pan comido esto de multiplicar matrices.

Pero apuesto a que a tus ojos verdes de ciencia ficción les gustaría más algo como esto:

$$
\mathbf{L} = \lambda \mathbf{\Omega}
$$

Que los dos vectores sean paralelos y así todo sea fácil otra vez. Como en los buenos tiempos, cuando $I$ era diagonal...

En este caso $\mathbf{L}$ sería constante.

<br>

> Así pues, ¿en qué dirección tendría que rotar la caja para que el momento angular sea constante?

Es decir, que vectores $\mathbf{\Omega}$ cumplen

$$
\lambda \mathbf{\Omega} = I \mathbf{\Omega}
$$

<br>

<br>

<br>

<br>

Dejo que repose.

<br>

<br>

<br>

¿Lo ves ya?

Te doy más tiempo...

<br>

<br>

<br>

<br>

¡Pues claro!

Esa ecuación nos está diciendo que si $\mathbf{\Omega}$ es un autovector de la matriz de inercia, entonces $\mathbf{L}$ será constante. _Mmmm yummy._

En este caso la solución es obvia. Basta con coger una de estas tres opciones:

$$
\mathbf{\Omega}
=\begin{pmatrix}
\omega \\
0 \\
0
\end{pmatrix}
$$

$$
\mathbf{\Omega}
=\begin{pmatrix}
0 \\
\omega \\
0
\end{pmatrix}
$$

$$
\mathbf{\Omega}
=\begin{pmatrix}
0 \\
0 \\
\omega
\end{pmatrix}
$$

Por ejemplo, para el primer caso, la rotación queda así:

(ahora fíjate que la flechita está fija)

In [9]:
omega = np.array([1,0,0])
L0 = I0 @ omega

In [10]:
fig = plt.figure()
ax = fig.add_subplot(projection="3d")

# Define the initial lines for the edges
lines = [ax.plot(*vertices[edge,:].T,
                 color='black')[0] for edge in edges]

markers = [ax.plot(*vert,
                 color='black', marker='.',
                markersize=10)[0] for vert in vertices]

arrows = [ax.quiver(0,0,0,L0[0],L0[1],L0[2], alpha=0.5)]

# Setting the axes properties
ax.set(xlim3d=(-2, 2), xlabel='X')
ax.set(ylim3d=(-2, 2), ylabel='Y')
ax.set(zlim3d=(-2, 2), zlabel='Z')

# Set up the animation
anim = animation.FuncAnimation(
    fig, update_mesh, 360,
    fargs=(lines,markers, arrows), interval=25)

video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()

## ¿Y si la matriz es MÁS complicada?
## La cafeína bailando pole dance

Pongamos que tenemos un objeto más complicado, la molécula de la cafeína.

Una moleculón de nombre completo _1,3,7-trimetil- 1H-purina- 2,6(3H,7H)-diona
1,3,7-trimetilxantina_.

La semana pasada te pedía las posiciones de los átomos en esta molecula. Ya sabes para qué.

Voy a cargar los datos y calcular la matriz de inercia, ya que estoy, también la voy a dibujar bailando:

In [11]:
# Load the xyz file
import pandas as pd

moleculepd = pd.read_table("data/cafeine.xyz", skiprows=2,
                           delim_whitespace=True,
                         names=['atom', 'x', 'y', 'z'])

atoms = moleculepd['atom'].to_numpy()

# Load position of the atoms
positions = moleculepd[['x','y','z']].to_numpy()

#Read links
links = np.genfromtxt('data/cafeine.links', delimiter=' ', dtype=int)

# Asign the mass of each atom
mass = {'H' : 1,
        'C' : 12,
        'N' : 7,
        'O' : 16}

In [12]:
# Center the atoms to the mass center
mc = np.average(positions,
                weights=list(map(lambda x: mass[x], atoms)),
               axis=0)
positions = positions - mc

# Compute the inertia matrix wrt mc
I0 = np.sum([mass[a] * np.array([[r[1]**2+r[2]**2, -r[0]*r[1], -r[0]*r[2]],
                                [-r[0]*r[1], r[0]**2+r[2]**2, -r[1]*r[2]],
                                [-r[0]*r[2], -r[1]*r[2], r[0]**2+r[1]**2]])
            for a, r in zip(atoms, positions)], axis=0)
I0

array([[ 4.75267180e+02,  1.05610068e+00,  9.81036452e+01],
       [ 1.05610068e+00,  1.05373615e+03, -3.79107380e-01],
       [ 9.81036452e+01, -3.79107380e-01,  5.88291065e+02]])

In [13]:
# Asign a color to each atom
colors = {'H': (.7,.7,.7),
          'C': (.1,.1,.1),
          'N': (0,0,1),
          'O': (1,0,0)}

# Vertices are atoms
vertices = positions

# Edges are the bonds
edges = links


# Redefine omega
omega = np.array([0, 1, 1])

# Compute the initial angular momentum
L0 = I0 @ omega

In [14]:
fig = plt.figure()
ax = fig.add_subplot(projection="3d")

# Define the initial lines for the edges

lines = [ax.plot(*vertices[edge,:].T,
                 color='black',
                markersize=30)[0] for edge in edges]

markers = [ax.plot(*pos,
                 color='black', mfc=colors[a], mec=colors[a],
                 marker='.',
                markersize=30)[0] for a, pos in zip(atoms, positions)]

arrows = [ax.quiver(0,0,0,L0[0],L0[1],L0[2], alpha=0.5)]

# Setting the axes properties
ax.set(xlim3d=(-5, 5), xlabel='X')
ax.set(ylim3d=(-5, 5), ylabel='Y')
ax.set(zlim3d=(-5, 5), zlabel='Z')

# Set up the animation
anim = animation.FuncAnimation(
    fig, update_mesh, 360,
    fargs=(lines, markers, arrows), interval=25)

video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()



Para simplificarnos la vida, nos gustaría encontrar esos ejes para los cuales $I$ es diagonal.

Ese es el tipo de brujería que podemos hacer con la diagonalización.


<br>

La clave es que si diagonalizas la matriz de inercia sus autovalores te darán $I_1$, $I_2$ e $I_3$ mientras que $\omega_1$, $\omega_2$ y $\omega_3$ son los componentes del vector $\Omega$ descrito en esa base.

Es decir,

$$
I = X
\begin{pmatrix}
I_1 & & \\
 & I_2 & \\
 & & I_3
\end{pmatrix}
X^{-1}
$$

$$
\begin{pmatrix}
\omega_1 \\
\omega_2 \\
\omega_3
\end{pmatrix}
= X
\Omega
$$

Podemos sacar esos ejes con la función `eig` de `scipy.linalg`. Esta función devuelve un vector con los elementos diagonales de $D$ y la matriz $X$.

Basta con escoger $\mathbf{\Omega}$ paralelo a cualquiera de las columnas de $X$.

In [15]:
from scipy import linalg
d, X = linalg.eig(I0)

omega = X[:,0]
L0 = I0 @ omega

In [16]:
fig = plt.figure()
ax = fig.add_subplot(projection="3d")

# Define the initial lines for the edges

lines = [ax.plot(*vertices[edge,:].T,
                 color='black',
                markersize=30)[0] for edge in edges]

markers = [ax.plot(*pos,
                 color='black', mfc=colors[a], mec=colors[a],
                 marker='.',
                markersize=30)[0] for a, pos in zip(atoms, positions)]

arrows = [ax.quiver(0,0,0,L0[0],L0[1],L0[2], alpha=0.5)]

# Setting the axes properties
ax.set(xlim3d=(-5, 5), xlabel='X')
ax.set(ylim3d=(-5, 5), ylabel='Y')
ax.set(zlim3d=(-5, 5), zlabel='Z')

# Set up the animation
anim = animation.FuncAnimation(
    fig, update_mesh, 360,
    fargs=(lines, markers, arrows), interval=25)

video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()

Como ves, la flechita ya no gira.

El momento angular no cambia.

El vector de rotación y el momento angular son paralelos.

Y toda esta magia ocurre cuando $\mathbf{\Omega}$ es un autovector de $I$.

Esto es importante para entender los procesos en los que la rotación de las moléculas juegan un papel, por ejemplo.

<br>

Como puedes ver, la diagonalización hace bailar hasta a la cafeína.

Así que, disfrútala tú también.